In [1]:
import transformers
import torch
transformers.BERT_PRETRAINED_MODEL_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/pytorch_model.bin"
transformers.BERT_PRETRAINED_CONFIG_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/config.json"
transformers.tokenization_bert.PRETRAINED_VOCAB_FILES_MAP["vocab_file"]["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/vocab.txt"
transformers.tokenization_bert.PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES["bert-base-finnish-cased"]=512
transformers.tokenization_bert.PRETRAINED_INIT_CONFIGURATION["bert-base-finnish-cased"]={'do_lower_case': False}

print("CUDA:",torch.cuda.is_available())

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


CUDA: True


In [2]:
import gen_finetune_dataset as ds

In [3]:
model=transformers.BertForMaskedLM.from_pretrained("bert-base-finnish-cased")
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-finnish-cased")

In [4]:
model=model.cuda()

/home/ginter/venv-transformers/lib/python3.5/site-packages/torch/cuda/__init__.py:118: UserWarning: 
    Found GPU1 Quadro K600 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [5]:
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50105, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
    

In [6]:
import gzip
f=gzip.open("delivery.0.25_0.txt.conllu.gz","rt")
sent_examples=ds.sent_examples_from_conllu(f)


In [7]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer=transformers.optimization.AdamW(optimizer_grouped_parameters,lr=0.001)
scheduler = transformers.WarmupLinearSchedule(optimizer, warmup_steps=100, t_total=10000)
model.zero_grad()

In [8]:
for idx,x in enumerate(ds.batch(sent_examples, tokenizer)):
    print("MEM=",torch.cuda.get_device_properties(0).total_memory)
    if idx<1:
        inp,mask,outp=x
    model.train()
    inp=inp.cuda()
    mask=mask.cuda()
    outp=outp.cuda()
    outputs=model(inp,attention_mask=mask,masked_lm_labels=outp)
    loss=outputs[0]
    print(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()
    model.zero_grad()
    
    

MEM= 8508604416
14.4618501663208
MEM= 8508604416
13.83191967010498
MEM= 8508604416
12.338517189025879
MEM= 8508604416
9.738017082214355
MEM= 8508604416
7.719480514526367
MEM= 8508604416
3.96358323097229
MEM= 8508604416
1.7148561477661133
MEM= 8508604416
2.1614792346954346
MEM= 8508604416
2.113901138305664
MEM= 8508604416
1.5068615674972534
MEM= 8508604416
1.0963014364242554
MEM= 8508604416
1.0095512866973877
MEM= 8508604416
0.7150914669036865
MEM= 8508604416
0.6003246903419495
MEM= 8508604416
0.5225520730018616
MEM= 8508604416
0.3864087462425232
MEM= 8508604416
0.3487587869167328
MEM= 8508604416
0.21566660702228546
MEM= 8508604416
0.2063206136226654
MEM= 8508604416
0.20925022661685944
MEM= 8508604416
0.1812913566827774
MEM= 8508604416
0.158519446849823
MEM= 8508604416
0.15237319469451904
MEM= 8508604416
0.11542224884033203
MEM= 8508604416
0.48630183935165405
MEM= 8508604416
0.29011270403862
MEM= 8508604416
0.5506410598754883
MEM= 8508604416
0.445869505405426
MEM= 8508604416
0.394873380

KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()
print("MEM=",torch.cuda.get_device_properties(0).total_memory)

In [ ]:
loss.item()